In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
def theurl(name):
 answer={"2012":"http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H155", "2013":"http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H163", "population": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H157", "medical": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H154", "risk": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H140", "employment": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H131", "jobs": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H158", "person_round_plan": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H153"}
 return answer[name]

In [3]:
def make_soup(url):
 response=requests.get(url)
 soup=BeautifulSoup(response.content)
 return soup

In [4]:
this=make_soup(theurl("2013"))

Querying this parser traverses the tree:

And it looks like the variable names and column limits are all contained in bits of the code that look like:

    <font class="smallBlack" face="Arial">CANCER DIAGNOSED - SKIN-NONMELANO (&gt;17)</font>,
    <font class="smallBlack" face="Arial">363   </font>,
    <font class="smallBlack" face="Arial">364   </font>,

The next function extracts all the tags in the parse tree with attribute "font" and collects them in a list, that I (in very poor taste) called final. In my defense, the list is really nothing, and nothing is ever final, so there you go.

I then use a helper function I called "prune_list" to get rid of the crud I didn't want to deal with that also happened to be part of a "font" tag. Finally, this function writes the code dictionary table to the file provided as the second argument to the function.

In [5]:
def get_data(soup,target):
 result=[]
 for row in soup.find_all("font"):
  result.append(row.contents)
 final=[(term[0].replace(u'\xa0',u'')).replace(',','') for term in result]
# get rid of junk
 final=prune_list(final,"::")
 final=prune_list(final,"MEPS HC-155")
 final=prune_list(final,"MEPS HC-163")
 final=prune_list(final,"MEPS HC-157")
 final=prune_list(final,"MEPS H154 CODEBOOK")
 final=prune_list(final,"MEPS H140 CODEBOOK")
 final=prune_list(final,"MEPS H131 CODEBOOK")
 final=prune_list(final,"MEPS HC-150")
 final=prune_list(final,"MEPS HC-158")
 final=prune_list(final,"MEPS H153 CODEBOOK")
 final=prune_list(final,"2012 FULL YEAR CONSOLIDATED DATA CODEBOOK")
 final=prune_list(final,"2013 FULL YEAR CONSOLIDATED DATA CODEBOOK")
 final=prune_list(final,"2013 FULL YEAR POPULATION CHARACTERISTICS CODEBOOK")
 final=prune_list(final,"2012 MEPS MEDICAL CONDITIONS FILE")
 final=prune_list(final,"2002-2009 RISK ADJUSTMENT SCORES FILE")
 final=prune_list(final,"EMPLOYMENT VARIABLE IMPUTATION FILE")
 final=prune_list(final,"2012 JOBS FILE CODEBOOK")
 final=prune_list(final,"2013 JOBS FILE CODEBOOK")
 final=prune_list(final,"2012 PERSON ROUND PLAN FILE")
 final=prune_list(final,"DATE:   August 25 2014")
 final=prune_list(final,"DATE:   August 25 2015")
 final=prune_list(final,"DATE: August 21 2014")
 final=prune_list(final,"DATE: August 4 2014")
 final=prune_list(final,"DATE:     March 6 2015")
 final=prune_list(final,"DATE: December 15 2014")
 final=prune_list(final,"DATE:    April 10 2013")
 final=prune_list(final,"DATE:   August 12 2014")
 final=prune_list(final,"DATE: February 13 2015")
 done=final
 with open(target,"w") as f:
  f.write("start,end,variable\n")
  for skip in range(0,len(final)-1,3):
   f.write("%s,"%final[skip])
   f.write("%s,"%final[skip+1])
   f.write("%s\n"%((final[skip+2]).lstrip()).rstrip())
 print("Done")
 temp=pd.read_csv(target)
 temp=temp.sort("start").copy()
 temp.index=list(range(len(temp)))
 return temp


Here's the definition of prune_list for reference:

In [6]:
def prune_list(thelist, theterm):
 while True:
  try:
   thelist.pop(thelist.index(theterm))
  except ValueError:
   break
 return thelist

Finally, here's the function that takes the MEPS dataset, and a dataframe containing the column dictionary, and writes the dataset out into a comma separated values file:

In [7]:
def write_table(data,header,target,short="No"):
 if short!="No":
  data=data[:10]
 data=prune_list(data,'')
 header=header.sort("start").copy()
 header.index=list(range(len(header)))
 with open(target,"w") as f:
  for element in header["variable"]:
   f.write("%s, "%element)
  f.write("\n")
 for row in data:
  u=[row[(header["start"].iloc[place]-1):(header["end"].iloc[place])] for place in header.index]
  with open(target,"a") as f:
   writer=csv.writer(f)
   writer.writerow(u)
  #print("Done with row %s"%data.index(row))


The next thing to do is to take a look at the dataset we have. Let's load it as a pandas dataframe and see what it contains:

In [8]:
cd ../../data/

/home/soumya/research/insight/insight_project/data


In [9]:
ls

2013_consolidated_frame.csv  2013_validate_with_dummies.pkl  Untitled.ipynb
2013_test_with_dummies.pkl   2013_with_dummies_.pkl
2013_train_with_dummies.pkl  2013_with_dummies.pkl


In [10]:
data=pd.read_csv("./2013_consolidated_frame.csv")

First let's take a look at what features the dataset contains:

In [11]:
data.columns

Index(['DWELLING UNIT ID', ' PERSON NUMBER', ' PERSON ID (DUID + PID)',
       ' PANEL NUMBER', ' FAMILY ID (STUDENT MERGED IN) - R3/1',
       ' FAMILY ID (STUDENT MERGED IN) - R4/2',
       ' FAMILY ID (STUDENT MERGED IN) - R5/3',
       ' FAMILY ID (STUDENT MERGED IN) - 12/31/13',
       ' ANNUAL FAMILY IDENTIFIER', ' CPSFAMID', 
       ...
       ' TOTAL RX-PRV & TRI AMT 13', ' TOTAL RX-OTH COMBINED AMT 13',
       ' FINAL PERSON WEIGHT 2013', ' FINAL FAMILY WEIGHT 2013',
       ' POV ADJ FAMILY WGT-CPS FAM ON 12/31/13',
       ' FINAL SAQ PERSON WEIGHT 2013',
       ' FINAL DIABETES CARE SUPPLEMENT WEIGHT',
       ' VARIANCE ESTIMATION STRATUM - 2013',
       ' VARIANCE ESTIMATION PSU - 2013', ' '],
      dtype='object', length=1792)

The first thing I get from this is that I need to learn what these things mean. I'll probably have to look up the MEPS survey methodology or documentation in order to figure out what these features actually are.

The second thing I get out of looking at the column names is that they really do need a bit of cleanup. For example, the first feature is called "DWELLING UNIT ID", but the second is called " PERSON NUMBER" (note the extra space at the beginning). This seems to be a running problem in the column names.

How many of these column names do I need to fix?

In [12]:
len(data.columns)

1792

Ok, that's far too many to clean up by hand.

On the other hand, it looks like the cleanup consists of a couple of simple rules:

1. Remove the initial space in the feature name
2. Pop the last entry from the column names

Let's write a function to do the first job:

In [13]:
def clean_columns(theframe):
 temp=theframe.columns.map(lambda x: (((str(x).lstrip()).rstrip()))).copy()
 theframe.columns=temp.copy()
 return theframe

In [14]:
clean_columns(data)

,DWELLING UNIT ID,PERSON NUMBER,PERSON ID (DUID + PID),PANEL NUMBER,FAMILY ID (STUDENT MERGED IN) - R3/1,FAMILY ID (STUDENT MERGED IN) - R4/2,FAMILY ID (STUDENT MERGED IN) - R5/3,FAMILY ID (STUDENT MERGED IN) - 12/31/13,ANNUAL FAMILY IDENTIFIER,CPSFAMID,...,TOTAL RX-PRV & TRI AMT 13,TOTAL RX-OTH COMBINED AMT 13,FINAL PERSON WEIGHT 2013,FINAL FAMILY WEIGHT 2013,POV ADJ FAMILY WGT-CPS FAM ON 12/31/13,FINAL SAQ PERSON WEIGHT 2013,FINAL DIABETES CARE SUPPLEMENT WEIGHT,VARIANCE ESTIMATION STRATUM - 2013,VARIANCE ESTIMATION PSU - 2013,
0,20004,101,20004101,17,A,A,A,A,A,A,...,0,0,6594.801768,6263.398994,6263.398994,7960.062940,0.000000,1109,1,NaN
1,20004,102,20004102,17,A,A,A,A,A,A,...,0,0,7144.183132,6263.398994,6263.398994,7971.337575,0.000000,1109,1,NaN
2,20004,103,20004103,17,A,A,A,A,A,A,...,77,0,5616.278666,6263.398994,6263.398994,0.000000,0.000000,1109,1,NaN
3,20005,101,20005101,17,A,A,A,A,A,A,...,0,53,4045.586998,6080.690177,6080.690177,5604.764494,0.000000,1109,2,NaN
4,20005,102,20005102,17,A,A,A,A,A,A,...,0,0,6901.505814,6080.690177,6080.690177,7781.609891,0.000000,1109,2,NaN
5,20005,103,20005103,17,A,A,A,A,A,A,...,0,0,7510.348195,6080.690177,0.000000,7913.308819,0.000000,1109,2,NaN
6,20006,101,20006101,17,A,A,A,A,A,A,...,0,0,1579.090762,1662.637092,1662.637092,2774.336686,0.000000,1045,2,NaN
7,20006,102,20006102,17,A,A,A,A,A,A,...,0,0,1754.557897,1662.637092,1662.637092,2775.057769,0.000000,1045,2,NaN
8,20006,103,20006103,17,A,A,A,A,A,A,...,0,0,1798.211205,1662.637092,1662.637092,0.000000,0.000000,1045,2,NaN
9,20006,104,20006104,17,A,A,A,A,A,A,...,0,0,2119.474804,1662.637092,1662.637092,3268.594748,0.000000,1045,2,NaN


So far so good; now I just need to get rid of the last column.

In [53]:
data1=data.copy()
temp=list(data1.columns)
temp.pop()
temp[-10:]
temp.append("nothing")

In [54]:
data1.columns=temp

In [55]:
data1.drop("nothing", axis=1)

,DWELLING UNIT ID,PERSON NUMBER,PERSON ID (DUID + PID),PANEL NUMBER,FAMILY ID (STUDENT MERGED IN) - R3/1,FAMILY ID (STUDENT MERGED IN) - R4/2,FAMILY ID (STUDENT MERGED IN) - R5/3,FAMILY ID (STUDENT MERGED IN) - 12/31/13,ANNUAL FAMILY IDENTIFIER,CPSFAMID,...,TOT RX-OTH UNCLASS SRCE AMT 13,TOTAL RX-PRV & TRI AMT 13,TOTAL RX-OTH COMBINED AMT 13,FINAL PERSON WEIGHT 2013,FINAL FAMILY WEIGHT 2013,POV ADJ FAMILY WGT-CPS FAM ON 12/31/13,FINAL SAQ PERSON WEIGHT 2013,FINAL DIABETES CARE SUPPLEMENT WEIGHT,VARIANCE ESTIMATION STRATUM - 2013,VARIANCE ESTIMATION PSU - 2013
0,20004,101,20004101,17,A,A,A,A,A,A,...,0,0,0,6594.801768,6263.398994,6263.398994,7960.062940,0.000000,1109,1
1,20004,102,20004102,17,A,A,A,A,A,A,...,0,0,0,7144.183132,6263.398994,6263.398994,7971.337575,0.000000,1109,1
2,20004,103,20004103,17,A,A,A,A,A,A,...,0,77,0,5616.278666,6263.398994,6263.398994,0.000000,0.000000,1109,1
3,20005,101,20005101,17,A,A,A,A,A,A,...,0,0,53,4045.586998,6080.690177,6080.690177,5604.764494,0.000000,1109,2
4,20005,102,20005102,17,A,A,A,A,A,A,...,0,0,0,6901.505814,6080.690177,6080.690177,7781.609891,0.000000,1109,2
5,20005,103,20005103,17,A,A,A,A,A,A,...,0,0,0,7510.348195,6080.690177,0.000000,7913.308819,0.000000,1109,2
6,20006,101,20006101,17,A,A,A,A,A,A,...,0,0,0,1579.090762,1662.637092,1662.637092,2774.336686,0.000000,1045,2
7,20006,102,20006102,17,A,A,A,A,A,A,...,0,0,0,1754.557897,1662.637092,1662.637092,2775.057769,0.000000,1045,2
8,20006,103,20006103,17,A,A,A,A,A,A,...,0,0,0,1798.211205,1662.637092,1662.637092,0.000000,0.000000,1045,2
9,20006,104,20006104,17,A,A,A,A,A,A,...,0,0,0,2119.474804,1662.637092,1662.637092,3268.594748,0.000000,1045,2


Great, that worked. Alright, let's roll that into a function:

In [59]:
def pop_final_column(data):
 temp=list(data.columns)
 temp.pop()
 temp.append("nothing")
 data.columns=temp
 data=data.drop("nothing", axis=1)
 return data

In [60]:
data=pop_final_column(data)

In [61]:
data.columns

Index(['DWELLING UNIT ID', 'PERSON NUMBER', 'PERSON ID (DUID + PID)',
       'PANEL NUMBER', 'FAMILY ID (STUDENT MERGED IN) - R3/1',
       'FAMILY ID (STUDENT MERGED IN) - R4/2',
       'FAMILY ID (STUDENT MERGED IN) - R5/3',
       'FAMILY ID (STUDENT MERGED IN) - 12/31/13', 'ANNUAL FAMILY IDENTIFIER',
       'CPSFAMID', 
       ...
       'TOT RX-OTH UNCLASS SRCE AMT 13', 'TOTAL RX-PRV & TRI AMT 13',
       'TOTAL RX-OTH COMBINED AMT 13', 'FINAL PERSON WEIGHT 2013',
       'FINAL FAMILY WEIGHT 2013', 'POV ADJ FAMILY WGT-CPS FAM ON 12/31/13',
       'FINAL SAQ PERSON WEIGHT 2013', 'FINAL DIABETES CARE SUPPLEMENT WEIGHT',
       'VARIANCE ESTIMATION STRATUM - 2013', 'VARIANCE ESTIMATION PSU - 2013'],
      dtype='object', length=1791)

Perfect.

Next, I'd like to learn a bit more about what these features are. The main thing to keep in mind is that having a large number of features will make our model very unwieldy. 

In the first instance, I'd really like the model to be **useful**, and if I have to collect 1700 features or so about a person before I can give them any useful feedback, this model won't be used by anyone.

Secondly, for the purposes of my project, I'm most concerned with what a much more restricted set of features say about billed charges. I'd really like to know how the demographic, economic, or idiosyncratic health status of a consumer drive cost, since these are the things that can inform how we choose to design the market for healthcare services.

